In [2]:
# 2  Imports
import numpy as np
import pandas as pd
from pathlib import Path

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, metrics

from sklearn.metrics import roc_auc_score, average_precision_score

In [3]:
# 1. Upload the files
from google.colab import files
uploaded = files.upload()
import pandas as pd

RAW_SW   = "solar_wind_data.csv"
RAW_GEO  = "GeomagneticandSolarIndicies.csv"
RAW_HESSI= "hessi_flare_list.csv"

# 2. Define the cadence
CADENCE = "5T"  # 5-minute

# 3.1 Solar-wind to datetime index
sw = pd.read_csv("solar_wind_data.csv")
sw["Date"] = pd.to_datetime(sw["YYYY"], format="%Y") + pd.to_timedelta(sw["DOY"] - 1, unit="D")
sw["DateTime"] = sw["Date"] + pd.to_timedelta(sw["HR"], unit="H") + pd.to_timedelta(sw["MN"], unit="m")
sw = sw.set_index("DateTime").sort_index()
sw = sw.drop(columns=["YYYY", "DOY", "HR", "MN", "Date"])

# 3.2 Resample to 5‑min means (ensures regular grid)
sw = sw.resample(CADENCE).mean()

# 3.3 Daily geomagnetic / solar indices
geo = pd.read_csv("GeomagneticandSolarIndicies.csv")
geo["Date"] = pd.to_datetime(geo[["Year", "Month", "Day"]].astype(str).agg("-".join, axis=1))
geo = geo.set_index("Date").sort_index()
geo_daily = geo[["Daily_Ap", "Sunspot_Number", "F10.7_Adj"]]

# Forward-fill geomagnetic data to 5‑min cadence
geo_5m = geo_daily.reindex(sw.index, method="ffill")

# 3.4 Merge datasets
df5 = sw.join(geo_5m, how="inner")

# 3.5 Create southward vBz proxy
df5["vBz_south"] = df5["Speed_km_s"] * df5["BZ_GSM_nT"].clip(upper=0)

print("5‑min dataframe shape:", df5.shape)

Saving GeomagneticandSolarIndicies.csv to GeomagneticandSolarIndicies.csv
Saving hessi_flare_list.csv to hessi_flare_list.csv
Saving solar_wind_data.csv to solar_wind_data.csv


<ipython-input-3-3f79e1cbe5be>:16: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  sw["DateTime"] = sw["Date"] + pd.to_timedelta(sw["HR"], unit="H") + pd.to_timedelta(sw["MN"], unit="m")
<ipython-input-3-3f79e1cbe5be>:21: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  sw = sw.resample(CADENCE).mean()


5‑min dataframe shape: (2630304, 10)


In [4]:
hessi = pd.read_csv(RAW_HESSI)
hessi["Start_DT"] = pd.to_datetime(hessi["Start_Time"])
flare_series = (hessi.assign(flag=1)
                       .set_index("Start_DT")
                       .resample(CADENCE)["flag"]
                       .max()
                       .fillna(0))

# Look‑ahead 24 h label
# --- FINAL NaN‑removal patch ---------------------------------------------
label = (flare_series
         .rolling("24H").max()
         .shift(-24*60//5)        # look‑ahead 24 h
         .reindex(df5.index))

# 1️⃣  Replace NaNs with 0 **AND** cast to float32
label = label.fillna(0).astype("float32")

df5["flare_next_24h"] = label

# 2️⃣  Sanity check
assert not df5["flare_next_24h"].isna().any(), "Label still contains NaNs!"
print("✅  Label column is NaN‑free")
# -------------------------------------------------------------------------

POS_RATE = df5["flare_next_24h"].mean()
print(f"Positive rate (5‑min grid): {POS_RATE:.4%}")

<ipython-input-4-3e8fbe58e6cb>:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample(CADENCE)["flag"]
<ipython-input-4-3e8fbe58e6cb>:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .rolling("24H").max()


✅  Label column is NaN‑free
Positive rate (5‑min grid): 51.5763%


In [5]:
train_end = "2013-12-31 23:55"
val_end   = "2016-12-31 23:55"

train_df = df5.loc[:train_end]
val_df   = df5.loc[train_end:val_end].iloc[1:]
test_df  = df5.loc[val_end:].iloc[1:]

print("Train rows:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

Train rows: 1472832 Val: 315648 Test: 841824


In [6]:
# --- REPLACEMENT FOR STEP 6 -----------------------------------------------
LOOKBACK_STEPS = 576   # 48h @ 5‑min
STEP            = 1
TARGET_COL      = "flare_next_24h"

# 1️⃣  Select numeric features
feature_cols = [c for c in df5.columns if c not in [TARGET_COL] and
                np.issubdtype(df5[c].dtype, np.number)]

# 2️⃣  Replace ±Inf with NaN, then fill NaN with 0
df5_clean = df5.copy()
df5_clean[feature_cols] = df5_clean[feature_cols].replace([np.inf, -np.inf], np.nan)
nan_mask = df5_clean[feature_cols].isna().astype("float32")   # 1 = was NaN
df5_clean[feature_cols] = df5_clean[feature_cols].fillna(0)

# 3️⃣  OPTIONAL: append the missing‑value indicators
use_nan_mask = False   # <- set True if you want the model to learn missingness
if use_nan_mask:
    mask_cols = [f"{c}_nan" for c in feature_cols]
    df5_clean[mask_cols] = nan_mask
    feature_cols = feature_cols + mask_cols

print("Final feature count:", len(feature_cols))

# 4️⃣  Normalise (mean‑0, std‑1) on TRAIN ONLY
mean = df5_clean.loc[:train_end, feature_cols].mean()
std  = df5_clean.loc[:train_end, feature_cols].std().replace(0, 1)

df5_clean[feature_cols] = (df5_clean[feature_cols] - mean) / std

def make_dataset(dataframe, shuffle=False):
    x = dataframe[feature_cols].values.astype("float32")
    y = dataframe[TARGET_COL].values.astype("float32")
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        x, y,
        sequence_length=LOOKBACK_STEPS,
        sequence_stride=STEP,
        shuffle=shuffle,
        batch_size=256
    )
    return ds

train_ds = make_dataset(df5_clean.loc[:train_end], shuffle=True)
val_ds   = make_dataset(df5_clean.loc[train_end:val_end].iloc[1:])
test_ds  = make_dataset(df5_clean.loc[val_end:].iloc[1:])

print("Dataset batches – Train:", len(train_ds),
      "Val:", len(val_ds),
      "Test:", len(test_ds))
# ---------------------------------------------------------------------------

Final feature count: 10
Dataset batches – Train: 5752 Val: 1231 Test: 3287


In [7]:
def pr_auc(y_true, y_pred):
    # PR‑AUC via tf‑metrics (works in TF 2.11+)
    return tf.keras.metrics.AUC(curve="PR", name="pr_auc")(y_true, y_pred)

inputs = layers.Input(shape=(LOOKBACK_STEPS, len(feature_cols)))
x = layers.Masking(mask_value=np.nan)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(32))(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[metrics.AUC(curve="ROC"), metrics.AUC(curve="PR", name="pr_auc")]
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 576, 10)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 576, 10)   │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking (Masking)   │ (None, 576, 10)   │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any (Any)           │ (None, 576)       │          0 │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 576, 128)  │     38,400 │ masking[0][0],    │
│ (Bidirectional)     │                   │            │ any[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 64)        │     41,216 │ bidirectional[0]… │
│ (Bidirectional)     │                   │            │ any[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      2,080 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         33 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 81,729 (319.25 KB)

 Trainable params: 81,729 (319.25 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices("GPU"))


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
early = callbacks.EarlyStopping(monitor="val_pr_auc",
                                mode="max",
                                patience=5,
                                restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=30,
    validation_data=val_ds,
    callbacks=[early]
)

Epoch 1/30
5752/5752 ━━━━━━━━━━━━━━━━━━━━ 814s 140ms/step - auc: 0.8411 - loss: 0.4401 - pr_auc: 0.9223 - val_auc: 0.4712 - val_loss: 0.7651 - val_pr_auc: 0.8091
Epoch 2/30
5752/5752 ━━━━━━━━━━━━━━━━━━━━ 803s 139ms/step - auc: 0.8815 - loss: 0.3859 - pr_auc: 0.9456 - val_auc: 0.5672 - val_loss: 0.9408 - val_pr_auc: 0.8523
Epoch 3/30
5752/5752 ━━━━━━━━━━━━━━━━━━━━ 807s 140ms/step - auc: 0.9420 - loss: 0.2703 - pr_auc: 0.9738 - val_auc: 0.5956 - val_loss: 1.3600 - val_pr_auc: 0.8585
Epoch 4/30
5752/5752 ━━━━━━━━━━━━━━━━━━━━ 866s 141ms/step - auc: 0.9719 - loss: 0.1921 - pr_auc: 0.9873 - val_auc: 0.5559 - val_loss: 1.9563 - val_pr_auc: 0.8404
Epoch 5/30
5752/5752 ━━━━━━━━━━━━━━━━━━━━ 858s 140ms/step - auc: 0.9891 - loss: 0.1209 - pr_auc: 0.9950 - val_auc: 0.5243 - val_loss: 2.4432 - val_pr_auc: 0.8293
Epoch 6/30
5752/5752 ━━━━━━━━━━━━━━━━━━━━ 861s 140ms/step - auc: 0.9951 - loss: 0.0784 - pr_auc: 0.9976 - val_auc: 0.5219 - val_loss: 2.5930 - val_pr_auc: 0.8270
Epoch 7/30
5752/5752 ━━━━━━━

In [10]:
from google.colab import files
import os

# 1. Create a directory to hold both formats
MODEL_DIR = "saved_models"
os.makedirs(MODEL_DIR, exist_ok=True)

# 2. Save in modern .keras format (recommended)
keras_path = f"{MODEL_DIR}/lstm_model_thisone.keras"
model.save(keras_path)

# 3. Save in legacy .h5 format (less reliable, but portable if needed)
h5_path = f"{MODEL_DIR}/lstm_model_thisone.h5"
model.save(h5_path, save_format="h5")  # explicitly set format to h5

# 4. Download both files
files.download(keras_path)
files.download(h5_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
test_pred = model.predict(test_ds).ravel()
test_true = np.concatenate([y for _, y in test_ds])

roc = roc_auc_score(test_true, test_pred)
pr  = average_precision_score(test_true, test_pred)
print(f"TEST  – ROC‑AUC: {roc:.3f}  PR‑AUC: {pr:.3f}")

3287/3287 ━━━━━━━━━━━━━━━━━━━━ 191s 58ms/step
TEST  – ROC‑AUC: 0.581  PR‑AUC: 0.101
